In [ ]:
%pip install tensorflow scikit-learn opencv-python tkinter matplotlib --quiet

In [ ]:
import numpy as np
import cv2
import tkinter as tk
from tkinter import filedialog
from tkinter import Label, Button, Canvas
from PIL import Image, ImageTk
import matplotlib.pyplot as plt
import joblib
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

In [ ]:
nationality_model = load_model('models/nationality_model.h5')
emotion_model = load_model('models/emotion_model.h5')
age_model = load_model('models/age_model.h5')
dress_color_model = joblib.load('models/dress_color_model.pkl')

In [ ]:
nationality_labels = ['Indian', 'American', 'African', 'Other']
emotion_labels = ['Happy', 'Sad', 'Angry', 'Neutral']
dress_color_labels = ['Red', 'Blue', 'Green', 'Yellow', 'White', 'Black']

In [ ]:
def preprocess_image(path):
    img = cv2.imread(path)
    img_resized = cv2.resize(img, (224, 224))
    img_array = img_to_array(img_resized) / 255.0
    return np.expand_dims(img_array, axis=0), img

In [ ]:
def predict_all(image_path):
    preprocessed, original_img = preprocess_image(image_path)
    
    nationality_idx = np.argmax(nationality_model.predict(preprocessed), axis=1)[0]
    emotion_idx = np.argmax(emotion_model.predict(preprocessed), axis=1)[0]
    
    nationality = nationality_labels[nationality_idx]
    emotion = emotion_labels[emotion_idx]
    
    age = None
    dress_color = None
    
    if nationality == 'Indian':
        age = int(age_model.predict(preprocessed)[0][0])
        dress_color = dress_color_model.predict(preprocessed.reshape(1, -1))[0]
    elif nationality == 'American':
        age = int(age_model.predict(preprocessed)[0][0])
    elif nationality == 'African':
        dress_color = dress_color_model.predict(preprocessed.reshape(1, -1))[0]
    
    return nationality, emotion, age, dress_color, original_img

In [ ]:
class App:
    def __init__(self, root):
        self.root = root
        self.root.title("Nationality Detection GUI")
        self.root.geometry("700x500")
        
        self.label = Label(root, text="Upload an image", font=("Arial", 14))
        self.label.pack()
        
        self.canvas = Canvas(root, width=300, height=300)
        self.canvas.pack()
        
        self.button = Button(root, text="Browse", command=self.browse_image)
        self.button.pack()
        
        self.output_label = Label(root, text="", font=("Arial", 12), wraplength=600)
        self.output_label.pack(pady=20)

    def browse_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            nationality, emotion, age, dress_color, original_img = predict_all(file_path)
            self.display_image(original_img)
            
            output_text = f"Nationality: {nationality}\nEmotion: {emotion}"
            if age is not None:
                output_text += f"\nAge: {age}"
            if dress_color is not None:
                output_text += f"\nDress Color: {dress_color}"
            
            self.output_label.config(text=output_text)

    def display_image(self, cv_img):
        img = cv2.cvtColor(cv_img, cv2.COLOR_BGR2RGB)
        img_pil = Image.fromarray(img)
        img_resized = img_pil.resize((300, 300))
        self.tk_image = ImageTk.PhotoImage(img_resized)
        self.canvas.create_image(0, 0, anchor='nw', image=self.tk_image)

In [ ]:
root = tk.Tk()
app = App(root)
root.mainloop()